In [0]:
/*produto: idProduto, descCategoria
pedido: idPedido, idCliente, dtPedido, dtAprovado, dtEnvio, dtEntregue
item_pedido: idPedido, idProduto, idPedidoItem, idVendedor, vlPreco, vlFrete
cliente: idCliente, descUF
avaliacao_pedido: idAvaliacao, idPedido, vlNota, dtAvaliacao
vendedor: idVendedor, descUF
pagamento_pedido: idPedido, descTipoPagamento, nrParcelas */

In [0]:

WITH tabela AS(
SELECT
      A.*,
      date(B.dtPedido),
      C.descUF
FROM silver.olist.item_pedido A

LEFT JOIN silver.olist.pedido B
ON A.idPedido = B.idPedido

LEFT JOIN silver.olist.vendedor C
ON A.idVendedor = C.idVendedor
)

SELECT 
      date(dtPedido) AS dtPedido,
      MONTH(dtPedido) AS Mes,
      YEAR(dtPedido) AS Ano,
      idVendedor,
      descUF,
      (CASE 
        WHEN descUF IN ('SP', 'RJ', 'MG', 'ES') THEN 'SUDESTE'
        WHEN descUF IN ('PR', 'SC', 'RS') THEN 'SUL'
        WHEN descUF IN ('RO', 'AC', 'AM', 'RR', 'AP', 'PA', 'TO') THEN 'NORTE'
        WHEN descUF IN ('MT', 'MS', 'GO', 'DF') THEN 'CENTRO-OESTE'
        WHEN descUF IN ('BA', 'SE', 'AL', 'CE', 'MA', 'PE') THEN 'NORDESTE'
      END) AS Regiao,
      SUM(vlPreco) + SUM(vlFrete) AS totalPreco
FROM tabela
WHERE date(dtPedido) < '2017-06-01'

GROUP BY ALL




In [0]:

WITH tabela AS(
SELECT
      A.*,
      date(B.dtPedido),
      C.descUF
FROM silver.olist.item_pedido A

LEFT JOIN silver.olist.pedido B
ON A.idPedido = B.idPedido

LEFT JOIN silver.olist.vendedor C
ON A.idVendedor = C.idVendedor
),

tabela2 AS(
SELECT 
      date(dtPedido) AS dtPedido,
      MONTH(dtPedido) AS Mes,
      YEAR(dtPedido) AS Ano,
      idVendedor,
      descUF,
      (CASE 
        WHEN descUF IN ('SP', 'RJ', 'MG', 'ES') THEN 'SUDESTE'
        WHEN descUF IN ('PR', 'SC', 'RS') THEN 'SUL'
        WHEN descUF IN ('RO', 'AC', 'AM', 'RR', 'AP', 'PA', 'TO') THEN 'NORTE'
        WHEN descUF IN ('MT', 'MS', 'GO', 'DF') THEN 'CENTRO-OESTE'
        WHEN descUF IN ('BA', 'SE', 'AL', 'CE', 'MA', 'PE') THEN 'NORDESTE'
      END) AS Regiao,
      SUM(vlPreco) + SUM(vlFrete) AS GMV
FROM tabela
WHERE date(dtPedido) < '2017-06-01'

GROUP BY ALL
),

shareGMV AS(
SELECT DISTINCT
      descUF,
      SUM(GMV) / (SELECT SUM(GMV) FROM tabela2) AS ShareGMV
FROM tabela2
GROUP BY ALL
)

SELECT
A.*,
B.shareGMV
FROM tabela2 A

LEFT JOIN shareGMV B
ON A.descUF = B.descUF



